<h1 align="center"> Projet Stats'App: Implémentation de LDPC </h1>

L'acronyme LDPC désigne les "low-density parity-check codes". Ces codes sont en fait des cas particuliers de codes correcteurs, qui sont nécessaires pour assurer une bonne transmission des données lors de transfert pouvant induire certaines perturbations. 

La partie codage consiste en la transformation du message composé de k bits en un mot codé sur n bits. On a donc n-k bits supplémentaires (dits de redondance), présents afin d'améliorer le décodage du code. Dans la suite, on notera G la matrice de codage. L'utilisation d'une telle matrice pour le codage vient du fait que le code est linéaire.  

> 
Le code est un sous-espace vectoriel de dimension k dit code. Une suite de bits x appartient au code si et seulement x s'écrit v.G, où v est le message de taille (1,k). Ou encore, si x n'appartient n'a pas de composante dans l'orthogonal du code, et donc en notant H une matrice formée par les lignes (e1,e2, ..en-k) (avec (e1, ..) base de l'orthogonal) , on a H.x' = 0. 
> 
La particularité des codes LDPC réside dans la construction de cette matrice H où le nombre de 1 doit croitre linéairement avec la taille du code. Le nombre de 1 dans la matrice H est donc dans ce cas relativement faible et la matrice est dite creuse. Trouver une telle matrice à partir d'un code n'est en général pas évident. On commmencera donc par construire une telle matrice de parité puis on déterminera la matrice génératrice G qui lui correspond. 







On importe tous les modules dont on aura besoin: 

In [25]:
import numpy as np
import math
from scipy import sparse 
from scipy.sparse import csr_matrix
import cv2
from time import time
from IPython.display import display, HTML
import scipy

 ## 1- Construction des matrices de décodage / codage
 ### 1- 1 Construction de la matrice de parité H
 
 #### Matrice de parité régulière 
 
On présente ci-dessous la méthode de construction de Callager d'une matrice de parité régulière avec la taille du message k, la taille du mot codé n, le nombre de 1 par ligne (resp. colonne)  d_c  (resp. d_v).

 Cette matrice doit donc vérifier: (n-k).d_c = n.d_v 

  > <h3 align="center">  Méthode de Callager </h3>
  
  > Dans l'image ci-dessous, K représente le nombre de 1 par ligne d_c, et w_c (c pour column) représente le nombre de 1 par colonne d_v.
  
  > <img src="Images/CallagerH.png">


In [26]:
def RegularH(n,d_v,d_c):

    """ ------------------------------------------------------------------------------

    Builds a regular Parity-Check Matrix H (n,d_v,d_c) following Callager's algorithm : 

    ----------------------------------------------------------------------------------

    Paramaeters:

     n: Number of columns (Same as number of coding bits)
     d_v: number of ones per column (number of parity-check equations including a certain variable) 
     d_c: number of ones per row (number of variables participating in a certain parity-check equation);  

    ----------------------------------------------------------------------------------

    Errors: 

     The number of ones in the matrix is the same no matter how we calculate it (rows or columns), therefore, if m is 
     the number of rows in the matrix: 

     m*d_c = n*d_v with m < n (because H is a decoding matrix) => Parameters must verify:


     0 - all integer parameters
     1 - d_v < d_v
     2 - d_c divides n 

    ---------------------------------------------------------------------------------------

     Returns: 2D-array (shape = (m,n))

    """


    if  n%d_c:
        raise ValueError('d_c must divide n. Help(RegularH) for more info.')

    if d_c <= d_v: 
        raise ValueError('d_c must be greater than d_v. Help(RegularH) for more info.')

    m = (n*d_v)// d_c
    a=m//d_v
    Set=np.zeros((a,n),dtype=int)  
    

    # Filling the first set with consecutive ones in each row of the set 

    for i in range(a):     
        for j in range(i*d_c,(i+1)*d_c): 
            Set[i,j]=1

    #Create list of Sets and append the first reference set
    Sets=[]
    Sets.append(Set.tolist())

    #Create remaining sets by permutations of the first set's columns: 
    i=1
    for i in range(1,d_v):
        newSet = np.transpose(np.random.permutation(np.transpose(Set))).tolist()
        Sets.append(newSet)

    #Returns concatenated list of sest:
    H = np.concatenate(Sets)
    return H


In [27]:

# Test de la fonction 
RegularH_test = 1

if RegularH_test:
    Htest = RegularH(12,3,4)
    print("La matrice de parité régulière (12,3,4):\n\n",Htest )

La matrice de parité régulière (12,3,4):

 [[1 1 1 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 1 1 1]
 [1 0 0 1 0 0 1 1 0 0 0 0]
 [0 0 0 0 0 1 0 0 1 1 0 1]
 [0 1 1 0 1 0 0 0 0 0 1 0]
 [1 1 0 0 0 0 0 0 0 0 1 1]
 [0 0 1 1 1 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 1 1 0 1 0 0]]


 ### 1-2 Construction de la Matrice de codage G à partir de H
 

### I - Le problème:
Soit une matrice de parité H donnée. Trouver la matrice de codage correspondante c'est résoudre l'équation en G avec G' transposée de G:
                                                
                                                H.G' = 0 (1)
                                                
                          Sous contrainte rang(G) = n - rang(H) (1')
                         
La contrainte (1') vient de:

<img src="Equations/Construction1.png">

> <font color=#A44057> rang(G) = dim(Ker(H)) </font> 

Par ailleurs, le théorème du rang appliqué à H donne: 
> <font color=#A44057> n = rang(H) + dim(ker(H)) </font> 
 
 D'où (1')
 
 
 ### II -  Solutions 
 
 
 
 On propose deux méthodes pour résoudre l'équation (1):
 
 - 1- En effectuant des opérations sur les lignes et les colonnes de H en utilisant les matrices équivalentes:
 on btient dans ce cas une matrice génératrice quelconque. 
 
 - 2- En <b> changeant H </b> (en permutant ses colonnes) pour trouver une matrice de codage associée <b> systématique </b> , c'est à dire sous la forme: 
 
<img src="Equations/Construction2.png">
 
 
 X représente la partie redondante de l'information. 

> <h3 align="center" font="b"> Solution 1: Matrice de Codage quelconque </h3>

### La théorie 

 En posant rang(G) = k, n-k est le rang H. Il existe P et Q inversibles de tailles respectives m (on rappelle que H a m lignes) et n telles que  P.H.Q = J où J est la matrice de taille m,n de rang n-k de la forme: 
                                        
 $$ \begin{pmatrix} I_{n-k} & 0_{n-k,k} \\ 0_{m-(n-k),n-k} & 0_{m-(n-k),k} \end{pmatrix} $$   
                                       
<img src="Equations/Construction3.png">

 On résout l'équation (2): 
 
<img src="Equations/Construction4.png">

 où Y1 est de taille (n-k,k) , et Y2 de taille (k,k). 
         
 
Le produit bloc par bloc de (2) donne : Y1 = 0 , Y2 quelconque. Or (1') impose que G soit de rang k. comme Y1 = 0 et Y2 est de taille k, Y2 doit être inversible, il suffit de prendre:  Y2 = I_k.  

 On trouve G = (Q.Y)' Où  
 
 <img src="Equations/Construction5.png">



###  L'Algorithme: 

On a besoin de plusieurs fonctions: 

- BinaryProduct: Fonction qui effectue le produit de deux matrices dans M_{p,q}(F_2) .

- GaussJordan: Fonction qui effectue les opérations sur les lignes d'une matrice A (forme échelonée réduite).
   
                                   |1 0 0 * * * * |
                                   |0 1 0 * * * * |
             Exemple:              |0 0 1 1 * * * |
                                   |0 0 0 0 1 * * |
                              
 Et renvoie aussi la matrice P telle que P.A = reduced(A) en effectuant les même opérations sur l'identité.
                                   
 Cette fonction effectue un double pivot de Gauss: On parcourt les colonnes et dès qu'un pivot est trouvé, tous les 1 au-dessus et au-dessous sont annulés par addition de lignes. voir l'algorithme de Gauss-Jordan. https://fr.wikipedia.org/wiki/%C3%89limination_de_Gauss-Jordan. 
 
Cette fonction sera appliquée à H pour les lignes et à sa transposée pour les colonnes. 
 
- CodingMatrix: Fonction qui calcule la matrice Q en effectuant un Gauss-Jordan double puis renvoie G = (Q.Y)'.
- BinaryRank: Fonction qui calcule le rang d'une matrice. (Basée sur Gauss-Jordan).


#### La fonction BinaryProduct: 

In [28]:
def BinaryProduct(X,Y):
        
    """ Binary Matrices or Matrix-vector product in Z/2Z. Works with scipy.sparse.csr_matrix matrices X,Y too."""
 
    A = X.dot(Y)
    
    if type(A)!=scipy.sparse.csr_matrix:
        return A%2
    
    return A.toarray()%2


#### La fonction GaussJordan:

In [29]:
def GaussJordan(MATRIX,change=0):

    """ 
    Description:

    Performs the row reduced echelon form of MATRIX and returns it.

    If change = 1, all changes in the MATRIX's rows are applied to identity matrix P: 

    Let A be our parameter MATRIX. refA the reduced echelon form of A. P is the square invertible matrix:

    P.A = Aref.

    -------------------------------------------------
    Parameters: 

    MATRIX: 2D-Array. 
    change : boolean (default = 0)

    ------------------------------------------------

    change = 0  (default)
     >>> Returns 2D-Array Row Reduced Echelon form of Matrix

    change = 1 
    >>> Returns Tuple of 2D-arrays (refMATRIX, P) where P is described above.

    """

    A = np.copy(MATRIX)
    m,n = A.shape
    

    if change:
        P = np.identity(m).astype(int)

    pivot_old = -1 
    for j in range(n):            

        filtre_down = A[pivot_old+1:m,j]
        pivot = np.argmax(filtre_down)+pivot_old+1
        

        if A[pivot,j]:
            pivot_old+=1 
            if pivot_old != pivot:
                aux = np.copy(A[pivot,:])
                A[pivot,:] = A[pivot_old,:]
                A[pivot_old,:] = aux
                if change:
                    aux = np.copy(P[pivot,:])
                    P[pivot,:] = P[pivot_old,:]
                    P[pivot_old,:] = aux

            

            for i in range(m):
                if i!=pivot_old and A[i,j]:
                    if change:
                        P[i,:] = abs(P[i,:]-P[pivot_old,:])
                    A[i,:] = abs(A[i,:]-A[pivot_old,:])


        if pivot_old == m-1:
            break

 
    if change:    
        return A,P 
    return A 


In [30]:
####################### Test de la fonction:

GaussJordan_test = 1

if GaussJordan_test:
    
    Htest=RegularH(20,4,5)
    
    #### On test GaussJordan sur Htest avec l'argument 1 pour obtenir la matrice de passage P:
    Hrowreduced, P = GaussJordan(Htest,1)
    print("\nLa matrice Hrowreduced P.H: \n\n{} \n ".format(Hrowreduced))

    #On vérife Hreducedtest = P.H 
    print("P.H = Hrowreduced ?",(BinaryProduct(P,Htest)==Hrowreduced).all())
    
    



La matrice Hrowreduced P.H: 

[[1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1]
 [0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]] 
 
P.H = Hrowreduced ? True


#### Le double Gauss-Jordan
Prenons une matrice de taille m,n avec n < m. Effectuer des opérations sur les lignes de H revient à multiplier à gauche H par une matrice P inversible (bien déterminée). Effectuer des opérations sur les colonnes de H c'est effectuer des opérations sur les lignes de sa transposée, et donc multiplier à gauche tH par une matrice tQ inversible. 

On commence par effectuer les opérations sur les colonnes, c'est-à-dire, on "GaussJordan" la transposée de H. Puis on GausseJordan la transposée de la matrice obtenue. Exemple:
                     

In [31]:
Exemple = 1
if Exemple:    
    Htest = RegularH(20,3,5)
    tHtest = np.transpose(Htest)
    print("\nLa matrice H:\n", Htest)
    print("\nLa matrice transposée de H:\n", tHtest)

    tHrowreduced = GaussJordan(tHtest)

    print("\nRowReduced appliqué à transposée de H (comme çi on manipulait les colonnes de H):\n",tHrowreduced)
    tHrowreducedT = np.transpose(tHrowreduced)
    print("\nPuis on transpose cette matrice:\n",tHrowreducedT)

    rowreducedtHrowreducedT = GaussJordan(tHrowreducedT)

    print("\n et on lui applique GaussJordan :\n",rowreducedtHrowreducedT)

    print(Htest.shape)


La matrice H:
 [[1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1]
 [1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 1]
 [0 0 0 1 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0]
 [1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 1 0 0 0]
 [0 0 1 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1]]

La matrice transposée de H:
 [[1 0 0 0 1 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 1 0 1 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 1 0]
 [1 0 0 0 0 1 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 1 0 0 0 1]
 [0 1 0 0 0 1 0 0 0 0 0 1]
 [0 1 0 0 0 1 0 0 0 0 0 1]
 [0 1 0 0 0 0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1 0 0 0 1 0]
 [0 1 0 0 0 0 1 0 0 1 0 0]
 [0 0 1 0 1 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 1 0 0]
 [0 0 1 0 0 1 0 0 0 0 1 0]
 [0 0 1 0 0 0 1 0 0 1 0 0]
 [0 0 0 1 0 0 0 1 0 0 1 0]
 [0 0

#### Fonction CodingMatrix 

In [32]:
def CodingMatrix(MATRIX,use_sparse=1):

    """ 
    CAUTION: RETURNS tG TRANSPOSED CODING MATRIX. 
    
    Function Applies GaussJordan Algorithm on Columns and rows of MATRIX in order
    to permute Basis Change matrix using Matrix Equivalence.

    Let A be the treated Matrix. refAref the double row reduced echelon Matrix.

    refAref has the form:

    (e.g) : |1 0 0 0 0 0 ... 0 0 0 0|  
            |0 1 0 0 0 0 ... 0 0 0 0| 
            |0 0 0 0 0 0 ... 0 0 0 0| 
            |0 0 0 1 0 0 ... 0 0 0 0| 
            |0 0 0 0 0 0 ... 0 0 0 0| 
            |0 0 0 0 0 0 ... 0 0 0 0| 


    First, let P1 Q1 invertible matrices: P1.A.Q1 = refAref

    We would like to calculate:
    P,Q are the square invertible matrices of the appropriate size so that:

    P.A.Q = J.  Where J is the matrix of the form (having MATRIX's shape):

    | I_p O | where p is MATRIX's rank and I_p Identity matrix of size p.
    | 0   0 |

    Therfore, we perform permuations of rows and columns in refAref (same changes
    are applied to Q1 in order to get final Q matrix)


    NOTE: P IS NOT RETURNED BECAUSE WE DO NOT NEED IT TO SOLVE H.G' = 0 
    P IS INVERTIBLE, WE GET SIMPLY RID OF IT. 
    
    Then
    
    solves: inv(P).J.inv(Q).G' = 0 (1) where inv(P) = P^(-1) and 
    P.H.Q = J. Help(PJQ) for more info.
    
    Let Y = inv(Q).G', equation becomes J.Y = 0 (2) whilst:
    
    J = | I_p O | where p is H's rank and I_p Identity matrix of size p.
        | 0   0 |
    
    Knowing that G must have full rank, a solution of (2) is Y = |  0  | Where k = n-p. 
                                                                 | I-k |
    
    Because of rank-nullity theorem. 
    
    -----------------
    parameters:
    
    H: Parity check matrix. 
    use_sparse: (optional, default True): use scipy.sparse format to speed up calculations
    ---------------
    returns:
    
    tG: Transposed Coding Matrix. 
    
    """


    H = np.copy(MATRIX)
    m,n = H.shape

    if m > n: 
        raise ValueError('MATRIX must have more rows than columns (a parity check matrix)')
    
    if n > 500 and use_sparse:
        sparse = 1
    
    else:
        sparse = 0
    ##### DOUBLE GAUSS-JORDAN:

    Href_colonnes,tQ = GaussJordan(np.transpose(H),1)

    Href_diag = GaussJordan(np.transpose(Href_colonnes))    

    Q=np.transpose(tQ)
    
    k = n - sum(Href_diag.reshape(m*n))

    
    Y = np.zeros(shape=(n,k)).astype(int)
    Y[n-k:,:] = np.identity(k)
    
    if sparse:
        Q = csr_matrix(Q)
        Y = csr_matrix(Y)

    tG = BinaryProduct(Q,Y)

    return tG


In [33]:
def BinaryRank(MATRIX):
    """ Computes rank of a binary Matrix using Gauss-Jordan algorithm"""
    A = np.copy(MATRIX)
    m,n = A.shape
    
    A = GaussJordan(A)
    
    return sum([a.any() for a in A])

In [34]:
##### TEST DE LA FONCTION CODINGMATRIX:

CodingMatrix_test=1

if CodingMatrix_test:
    n = 100
    d_c = 4
    d_v = 3
    Htest = RegularH(n,d_v,d_c)
    tGtest = CodingMatrix(Htest)
    
    print("Condition H.G' = 0 (1) Vérifiée ?",(BinaryProduct(Htest,tGtest)==0).all())

    print("\nContrainte de rang (1') vérifiée ?", BinaryRank(tGtest)==n -BinaryRank(Htest) )


Condition H.G' = 0 (1) Vérifiée ? True

Contrainte de rang (1') vérifiée ? True


> <h3 align="center" font="b"> Solution 2: Matrice de Codage systématique </h3>

### La théorie:

 <u><b> Lemme: H sous la forme systémtique et de rang maximal: </b></u>
 
 H s'écrit :
 
<img src="Equations/Construction6.png">

 Alors la matrice systématique :
 
<img src="Equations/Construction7.png">
 
 convient.

 <u><b> Cas particulier: Matrice de rang maximal K:</b></u> 

 Si H est une matrice de parité de rang maximal m = n-k, alors, un simple Gauss-Jordan permet de l'écrire sous la forme: 

<img src="Equations/Construction8.png">

on note T la matrice inversible telle que:  refreduced(K) = TH  

Or pour obtenir G sous la forme systématique, il faut que le bloc identité soit à droite, il faut donc appliquer une permutation aux colonnes de TH pour obtenir: 

<img src="Equations/Construction9.png">
 
Si l'on note par P la matrice de cette permutation:

<img src="Equations/Construction10.png">

On rappelle que l'inverse de P est la transposée de P. 

Et donc

<img src="Equations/Construction11.png">

Et comme H_s est systématique, d'après le lemme une solution GP' l'est aussi. 
Posons G_s = GP'. 

Il vient: 

<img src="Equations/Construction12.png">

Et donc on obtient finalement: 

 -  Une matrice de parité H modifiée par l'inverse de la permutation P.
 -  Une matrice de codage G_s systématique. 

 <u><b> Cas général: Matrice de parité de Callager:</b></u> 


Comme H n'est pas forcément de rang maximal m, appliquer Gauss-Jordan à H donne une matrice de la forme:

<img src="Equations/Construction13.png">

Il faut effectuer des permutations sur les colonnes de refreduced(H) = T.H pour l'écrire sous la forme: 

<img src="Equations/Construction14.png">

On note P_1 la première matrice relative à cette permutation donc

<img src="Equations/Construction15.png">

Notez bien que H_{ss} s'écrit sous la forme du <b> cas particulier </b> avec des lignes nulles en plus.  

Ces lignes ne changent en rien l'équation (1), et lui appliquant la permutation 

<img src="Equations/Construction10.png">

les permutations se combinent en une seule matrice:

<img src="Equations/Construction16.png">

et comme H_s est systématique, d'après le lemme une solution GP' l'est aussi se construisant facilement à partir de H_s.

Posons G_s = GP'. 

Il vient:

<img src="Equations/Construction17.png">

Et donc on obtient finalement: 

 -  Une matrice de parité H modifiée par l'inverse de la permutation P = P_2.P_1.
 -  Une matrice de codage $G_s$ systématique que l'on construit avec H_s.
 
 ### L'algorithme:
 
La fonction CodingMatrix_systematic effectue toutes ces opérations: 

- Elle applique Gauss-Jordan à H. 
- Permute les dernières colonnes de la forme échelonée réduite de H pour obtenir le bloc identité d'abord à gauche.
- Applique la permutation sigma pour obtenir le bloc identité à droite. 
- Récupère la matrice de permutation globale P et applique son inverse (transposée) à H pour renvoyer Hp.
- Récupère le bloc A dans la forme systématique de H pour construire Gs. 
- Renvoie le couple Hp,Gs. 

In [35]:
def CodingMatrix_systematic(MATRIX,use_sparse = 1):

    """ 
    Description:

    Solves H.G' = 0 and finds the coding matrix G in the systematic form : [I_k  A] by applying permutations on MATRIX.
    
    CAUTION: RETURNS TUPLE (Hp,tGS) WHERE Hp IS A MODIFIED VERSION OF THE GIVEN PARITY CHECK MATRIX, tGS THE TRANSPOSED 
    SYSTEMATIC CODING MATRIX ASSOCIATED TO Hp. YOU MUST USE THE RETURNED TUPLE IN CODING AND DECODING, RATHER THAN THE UNCHANGED 
    PARITY-CHECK MATRIX H. 

    -------------------------------------------------
    Parameters: 

    MATRIX: 2D-Array. Parity-check matrix.
    use_sparse: (optional, default True): use scipy.sparse matrices to speed up calculations if n>100.

    ------------------------------------------------

    >>> Returns Tuple of 2D-arrays (Hp,GS):
        Hp: Modified H: permutation of columns (The code doesn't change)
        tGS: Transposed Systematic Coding matrix associated to Hp.

    """

    H = np.copy(MATRIX)
    m,n = H.shape
    
    if n>100 and use_sparse:
        sparse = 1
    else:
        sparse = 0 
        
    P1 = np.identity(n,dtype=int)
    
    Hrowreduced = GaussJordan(H)
    
    k = n - sum([a.any() for a in Hrowreduced ])

    ## After this loop, Hrowreduced will have the form H_ss : | I_(n-k)  A |
    permut = np.array(list(range(n)))

    while(True):
        zeros = [i for i in range(min(m,n)) if not Hrowreduced[i,i]]
        indice_colonne_a = min(zeros)
        list_ones = [j for j in range(indice_colonne_a+1,n) if Hrowreduced[indice_colonne_a,j] ]
        if not len(list_ones):
            break

        indice_colonne_b = min(list_ones)
        
        aux = np.copy(Hrowreduced[:,indice_colonne_a])
        Hrowreduced[:,indice_colonne_a] = Hrowreduced[:,indice_colonne_b]
        Hrowreduced[:,indice_colonne_b] = aux 
        
        aux = np.copy(P1[:,indice_colonne_a])
        P1[:,indice_colonne_a] = P1[:,indice_colonne_b]
        P1[:,indice_colonne_b] = aux
        
    ############ NOW, Hrowreduced has the form: | I_(n-k)  A | , the permutation above makes it look like : 
    ########### |A  I_(n-k)|
    
    P1 = P1.T
    identity = list(range(n))
    sigma = identity[n-k:]+identity[:n-k]
    
    P2 = np.zeros(shape=(n,n),dtype=int)
    P2[identity,sigma] = np.ones(n)
    
    if sparse:
        P1 = csr_matrix(P1)
        P2 = csr_matrix(P2)
        H = csr_matrix(H)

    P = BinaryProduct(P2,P1)
    
    if sparse:
        P = csr_matrix(P)
        
    Hp = BinaryProduct(H,np.transpose(P))

    GS = np.zeros((k,n),dtype=int)
    GS[:,:k] = np.identity(k)
    GS[:,k:] = np.transpose(Hrowreduced[:n-k,n-k:])
    
    
    return Hp,np.transpose(GS)

In [36]:
Test_systematic = 1

if Test_systematic:
    n =100
    d_v = 3
    d_c = 4
    H = RegularH(n,d_v,d_c)
    t = time()
    Hp,tGS = CodingMatrix_systematic(H)
    k = BinaryRank(tGS)
    
    print("Condition Hp.Gs' = 0 (1) Vérifiée ?",(BinaryProduct(Hp,tGS)==0).all())

    print("\nContrainte de rang (1') vérifiée ?", k == n - BinaryRank(Hp) )
    
    print("\nGs est-elle systématique ?", (tGS[:k,:]==np.identity(k)).all() )


Condition Hp.Gs' = 0 (1) Vérifiée ? True

Contrainte de rang (1') vérifiée ? True

Gs est-elle systématique ? True


## 2- Codage et Décodage

Fixons le triplet (n,d_v,d_c). 
On commence par générer une matrice de parité H régulière (n,d_v,d_c) de taille(m,n)  avec la méthode de Callager. On obtient la matrice de codage G avec la fonction CodingMatrixG ci-dessus. On récupère k en accédant à la taille de G. 
    

### 2-1 CODAGE 

Prenons un message (suite de bits) aléatoire v de taille k. Le code transmis dans le canal est donc d=v.G . En appliquant la modulation BPSK, on travaille avec le vecteur émis de k-ième coordonnée x_k=(-1)^{d_k}. Une fois transmis dans le canal, on reçoit à l'autre bout un vecteur x + e où est un bruit blanc gaussien (si l'on suppose que le bruit du canal est additif). 

On définit le <i> Signal-Noise-Ratio (SNR)</i> par: 
Pour un bruit gaussien (toujours centré en 0) et de variance <img src="Equations/Construction18.png">

Le SNR est défini ci-dessus en décibels.

On présente deux fonctions de codage: 

- La fonction Coding_random génère aléatoirement un message de taille k et le code. 

- La fonction Coding code un message v de taille k passé en argument. 

In [37]:
def Coding_random(tG,SNR):
    """
    IMPORTANT: tG can be transposed coding matrix scipy.sparse.csr_matrix object to speed up calculations.

    Randomly computes a k-bits message v, where G's shape is (k,n). And sends it
    through the canal.
    
    Message v is passed to G: d = v,G. d is a n-vector turned into a BPSK modulated
    vector x. Then Additive White Gaussian Noise (AWGN) is added. 
    
    SNR is the Signal-Noise Ratio: SNR = 10log(1/variance) in decibels, where variance is the variance of the AWGN.
    Remember: 
        1. d = v.G => tG.tv = td
        2. x = BPSK(d) (or if you prefer the math: x = pow(-1,d) )
        3. y = x + AWGN(0,snr) 
    
    ----------------------------

    Parameters:

    tG: 2D-Array (OR scipy.sparse.csr_matrix) Transpose of Coding Matrix obtained from CodingMatrix functions.
    
    SNR: the Signal-Noise Ratio: SNR = 10log(1/variance) in decibels. 
        >> SNR = 10log(1/variance) 
    
    -------------------------------

    Returns

    Tuple(v,y) (Returns v to keep track of the random message v)
    
    """
    n,k = tG.shape
    
    v=np.random.randint(2,size=k)

    d = BinaryProduct(tG,v)
    x=pow(-1,d)

    sigma = 10**(-SNR/20)
    
    e = np.random.normal(0,sigma,size=n)
 
    y=x+e

    return(v,y)




In [38]:

def Coding(tG,v,SNR):
    """
    
    IMPORTANT: if H is large, tG (transposed coding matrix) can be  scipy.sparse.csr_matrix object to speed up calculations.

    Codes a message v with Coding Matrix G, and sends it through a noisy (default)
    channel. 
    
    G's shape is (k,n). 

    Message v is passed to tG: d = tG.tv d is a n-vector turned into a BPSK modulated
    vector x. Then Additive White Gaussian Noise (AWGN) is added. 
    
    SNR is the Signal-Noise Ratio: SNR = 10log(1/variance) in decibels, where variance is the variance of the AWGN.
    Remember: 
    
        1. d = v.G (or (td = tG.tv))
        2. x = BPSK(d) (or if you prefer the math: x = pow(-1,d) )
        3. y = x + AWGN(0,snr) 

    
    Parameters:

    tG: 2D-Array (OR scipy.sparse.csr_matrix)Transposed Coding Matrix obtained from CodingMatrix functions.
    v: 1D-Array, k-vector (binary of course ..) 
    SNR: Signal-Noise-Ratio: SNR = 10log(1/variance) in decibels. 

    -------------------------------

    Returns y

    """
    n,k = tG.shape

    if len(v)!= k:
        raise ValueError(" Size of message v must be equal to number of Coding Matrix G's rows " )  
    
    d = BinaryProduct(tG,v)
    x=pow(-1,d)

    sigma = 10**(-SNR/20)
    e = np.random.normal(0,sigma,size=n)


    y=x+e

    return y
   


In [39]:
########### TEST DES FONCTIONS DE CODAGE:
CODING_test = 1

if CODING_test:
    ntest =100
    d_vtest = 3
    d_ctest = 4
    Htest = RegularH(ntest,d_vtest,d_ctest)
    Htest,tGtest = CodingMatrix_systematic(Htest)
    tGsparse = csr_matrix(tGtest)
    
    print("TEST DE CODAGE, SNR = 0 (Variance =1)\n")
    print("CODAGE ALEATOIRE \n")
    print("Le message aléatoire envoyé est:\n {} \n \n \
    Le vecteur reçu y après transmission:\n {}".format(Coding_random(tGtest,0)[0],Coding_random(tGsparse,0)[1]))
    print("\n _____________________________________________________________ \n CODAGE SPECIFIQUE:\n")
    ktest = tGtest.shape[1]
    vtest = np.random.randint(2,size=ktest)

    print("Le message à k bits vtest donné est:\n {} \n \n \
    Le vecteur reçu y après transmission:\n {}".format(vtest,Coding(tGsparse,vtest,0)))
    


TEST DE CODAGE, SNR = 0 (Variance =1)

CODAGE ALEATOIRE 

Le message aléatoire envoyé est:
 [0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 0 0 1 0 1 0 0] 
 
     Le vecteur reçu y après transmission:
 [ 0.84356401  0.8485863  -0.67503432  1.13481934 -2.63565612 -1.26209385
  0.95918682 -1.28426828  2.07781204  1.15412043  0.09061849 -0.08333776
 -1.07985257 -0.87670084  1.42841226 -2.85720088 -1.03686701 -0.24959313
 -0.96140121  0.63464732  3.06917118 -0.28327719  1.25665331 -2.14365771
 -1.98529827 -2.01421305  1.05257607 -0.83013099  1.26012478 -2.54079882
  0.0842997   1.31318741 -0.0257579   0.73875371  0.90034267  1.63582258
  0.05511692 -0.17850981  0.51879047  0.26590847  2.14331619  0.38223382
  0.89601517  0.89529574 -0.937976    2.76909374 -2.01156082  1.34676893
  0.82940958  0.85393581 -2.76727851 -0.51531004 -0.73242723  0.13610703
  1.2730714   0.74358222  0.42102699 -0.98415196 -3.21547971 -0.66723172
  0.8687968  -0.49842351 -0.26925289  2.17250146 -1.3305789   1.76958569
 -

## 2-2 Décodage

### 2 - 2 -1 Algorithme BP

On introduit d'abord une série de fonctions qui serviront dans les fonctions de décodage.

In [40]:
pi = math.pi
def f1(y,sigma):
    """ Normal Density N(1,sigma) """ 
    return(np.exp(-.5*pow((y-1)/sigma,2))/(sigma*math.sqrt(2*pi)))

def fM1(y,sigma):
    """ Normal Density N(-1,sigma) """ 

    return(np.exp(-.5*pow((y+1)/sigma,2))/(sigma*math.sqrt(2*pi)))

def Bits2i(H,i):
    """
    Computes list of elements of N(i)-j:
    List of variables (bits) connected to Parity node i.

    """
    if type(H)!=scipy.sparse.csr_matrix:
        m,n=H.shape
        return ([a for a in range(n) if H[i,a] ])
    
    indj = H.indptr
    indi = H.indices
    
    return [indi[a] for a in range(indj[i],indj[i+1])]


def Nodes2j(tH,j):
    
    """
    Computes list of elements of M(j):
    List of nodes (PC equations) connecting variable j.

    """
    
    return Bits2i(tH,j)

def InCode(H,x):

    """ Computes Binary Product of H and x. If product is null, x is in the code.

        Returns appartenance boolean. 
    """
        
    return  (BinaryProduct(H,x)==0).all()

def BitsAndNodes(H):
    
    m,n = H.shape
    if type(H)==scipy.sparse.csr_matrix:
        tH = scipy.sparse.csr_matrix(np.transpose(H.toarray()))
    else:
        tH = np.transpose(H)
        
    Bits = [Bits2i(H,i) for i in range(m)]
    Nodes = [Nodes2j(tH,j)for j in range(n)]
    
    return Bits,Nodes


#### FONCTION DECODAGE - ALGORITHME BP

In [41]:
def Decoding_BP(H,y,SNR,max_iter=1):

    """ Decoding function using Belief Propagation algorithm.
        
        
        IMPORTANT: H can be scipy.sparse.csr_matrix object to speed up calculations if n > 1000 highly recommanded. 
    -----------------------------------
    Parameters:
    
    H: 2D-array (OR scipy.sparse.csr_matrix object) Parity check matrix, shape = (m,n) 

    y: n-vector recieved after transmission in the channel. (In general, returned 
    by Coding Function)


    Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.
    
    max_iter: (default = 1) max iterations of the main loop. Increase if decoding is not error-free.

     """
        
    m,n=H.shape
    if not len(y)==n:
        raise ValueError('Size of y must be equal to number of parity matrix\'s columns n')

    if m>=n:
        raise ValueError('H must be of shape (m,n) with m < n')

    sigma = 10**(-SNR/20)
    
    p0 = np.zeros(shape=n)
    p0 = f1(y,sigma)/(f1(y,sigma) + fM1(y,sigma))
    p1 = np.zeros(shape=n)
    p1 = fM1(y,sigma)/(f1(y,sigma) + fM1(y,sigma))


    #### ETAPE 0 : Initialization 
    q0 = np.zeros(shape=(m,n))
    q0[:] = p0

    q1 = np.zeros(shape=(m,n))
    q1[:] = p1

    r0 = np.zeros(shape=(m,n))
    r1 = np.zeros(shape=(m,n))

    count=0
    prod = np.prod
    
    Bits,Nodes = BitsAndNodes(H)

    
    while(True):

        count+=1 #Compteur qui empêche la boucle d'être infinie .. 

        #### ETAPE 1 : Horizontale

        deltaq = q0 - q1
        deltar = r0 - r1 
        

        for i in range(m):
            Ni=Bits[i]
            for j in Ni:
                Nij = Ni.copy()

                if j in Nij: Nij.remove(j)
                deltar[i,j] = prod(deltaq[i,Nij])
                

        r0 = 0.5*(1+deltar)
        r1 = 0.5*(1-deltar)


        #### ETAPE 2 : Verticale

        for j in range(n):
            Mj = Nodes[j]
            for i in Mj:
                Mji = Mj.copy()
                if i in Mji: Mji.remove(i)
                    
                q0[i,j] = p0[j]*prod(r0[Mji,j])
                q1[i,j] = p1[j]*prod(r1[Mji,j])
                
                if q0[i,j] + q1[i,j]==0:
                    q0[i,j]=0.5
                    q1[i,j]=0.5
              
                else:
                    alpha=1/(q0[i,j]+q1[i,j]) #Constante de normalisation alpha[i,j] 

                    q0[i,j]*= alpha
                    q1[i,j]*= alpha # Maintenant q0[i,j] + q1[i,j] = 1


        #### Calcul des probabilites à posteriori:
        q0_post = np.zeros(n)
        q1_post = np.zeros(n)
        
        for j in range(n):
            Mj=Nodes[j]
            q0_post[j] = p0[j]*prod(r0[Mj,j])
            q1_post[j] = p1[j]*prod(r1[Mj,j])
            
            if q0_post[j] + q1_post[j]==0:
                q0_post[j]=0.5
                q1_post[j]=0.5
                
            alpha = 1/(q0_post[j]+q1_post[j])
            
            q0_post[j]*= alpha
            q1_post[j]*= alpha
        

        x = np.array(q1_post > q0_post).astype(int)
        
        if InCode(H,x) or count >= max_iter:  
            break
  
    return x


### 2-2-2 Algorithme full-log BP 

In [42]:
def Decoding_logBP(H,y,SNR,max_iter=1):
    
    """ Decoding function using Belief Propagation algorithm (logarithmic version)

    IMPORTANT: if H is large (n>1000), H should be scipy.sparse.csr_matrix object to speed up calculations
    (highly recommanded. )
    -----------------------------------
    Parameters:
    
    H: 2D-array (OR scipy.sparse.csr_matrix object) Parity check matrix, shape = (m,n) 

    y: n-vector recieved after transmission in the channel. (In general, returned 
    by Coding Function)


    Signal-Noise Ratio: SNR = 10log(1/variance) in decibels of the AWGN used in coding.
    
    max_iter: (default = 1) max iterations of the main loop. Increase if decoding is not error-free.

     """
        
    m,n=H.shape

    if not len(y)==n:
        raise ValueError('La taille de y doit correspondre au nombre de colonnes de H')

    if m>=n:
        raise ValueError('H doit avoir plus de colonnes que de lignes')
    
    
    var = 10**(-SNR/10)

    ### ETAPE 0: initialisation 
    
    Lc = 2*y/var

    Lq=np.zeros(shape=(m,n))

    Lr = np.zeros(shape=(m,n))
    
    count=0
    
    prod=np.prod
    tanh = np.tanh
    log = np.log
    
    	
    Bits,Nodes = BitsAndNodes(H)
    while(True):

        count+=1 #Compteur qui empêche la boucle d'être infinie .. 

        #### ETAPE 1 : Horizontale
        for i in range(m):
            Ni = Bits[i]
            for j in Ni:
                Nij = Ni.copy()

                if j in Nij: Nij.remove(j)
            
                if count==1:
                    X = prod(tanh(0.5*Lc[Nij]))
                else:
                    X = prod(tanh(0.5*Lq[i,Nij]))
                num = 1 + X
                denom = 1 - X
                if num == 0: 
                    Lr[i,j] = -1
                elif denom  == 0:
                    Lr[i,j] =  1
                else: 
                    Lr[i,j] = log(num/denom)
              

        #### ETAPE 2 : Verticale
        for j in range(n):
            Mj = Nodes[j]
            
            for i in Mj:
                Mji = Mj.copy()
                if i in Mji: Mji.remove(i)

                Lq[i,j] = Lc[j]+sum(Lr[Mji,j])
        
 
        #### LLR a posteriori:
        L_posteriori = np.zeros(n)
        for j in range(n):
            Mj = Nodes[j]

            L_posteriori[j] = Lc[j] + sum(Lr[Mj,j])

        x = np.array(L_posteriori <= 0).astype(int)
            
        product = InCode(H,x)

        if product or count >= max_iter:  
            break
        
    return x

### 2-2-3 Réception du message à k bits après décodage
Après décodage et correction des erreurs du vecteur à n bits transmis, pour conclure quant à la réussite de la transmission, on doit pouvoir retrouver le message à k bits codé avec la matrice G à partir du vecteur décodé x. 
On sait que pour un vecteur donné x appartenant au code, il existe v vecteur à k bits tel que :
    
                        x = v . G  
Donc: 

                        G'.v' = x'

On propose de résoudre ce système en v en lui appliquant l'algorithme du pivot de Gauss codé ci-dessous dans la fonction GaussElimination. On sait que ce système a une solution car G est de rang k. Il s'agit d'un système de rang k à k inconnues. La solution existe et est unique.


In [43]:
def GaussElimination(MATRIX,B):

    """ Applies Gauss Elimination Algorithm to MATRIX in order to solve a linear system MATRIX.X = B. 
    MATRIX is transformed to row echelon form: 

         |1 * * * * * |
         |0 1 * * * * |
         |0 0 1 * * * |
         |0 0 0 1 * * | 
         |0 0 0 0 1 * |
         |0 0 0 0 0 1 |
         |0 0 0 0 0 0 |
         |0 0 0 0 0 0 |
         |0 0 0 0 0 0 |


    Same row operations are applied on 1-D Array vector B. Both arguments are sent back.
    
    --------------------------------------
    
    Parameters:
    
    MATRIX: 2D-array. 
    B:      1D-array. Size must equal number of rows of MATRIX.
            
    -----------------------------------
    Returns:
    
    Modified arguments MATRIX, B as described above.
    
         """

    A = np.copy(MATRIX)
    b = np.copy(B)
    n,k = A.shape


    if b.size != n:
        raise ValueError('Size of B must match number of rows of MATRIX to solve MATRIX.X = B')

    for j in range(min(k,n)):
        listeDePivots = [i for i in range(j,n) if A[i,j]]
        if len(listeDePivots)>0:
            pivot = np.min(listeDePivots)
        else:
            continue
        if pivot!=j:
            aux = np.copy(A[j,:])
            A[j,:] = A[pivot,:]
            A[pivot,:] = aux

            aux = np.copy(b[j])
            b[j] = b[pivot]
            b[pivot] = aux

        for i in range(j+1,n):
            if A[i,j]:
                A[i,:] = abs(A[i,:]-A[j,:])
                b[i] = abs(b[i]-b[j])

    return A,b




La fonction DecodedMessage applique l'algorithme du pivot de Gauss au système linéaire précédent et retrouve les k bits du message v en remontant. 

In [44]:
def DecodedMessage(tG,x):

    """
    Let G be a coding matrix. tG its transposed matrix. x a n-vector received after decoding.
    DecodedMessage Solves the equation on k-bits message v:  x = v.G => G'v'= x' by applying GaussElimination on G'.
    
    -------------------------------------
    
    Parameters:
    
    tG: Transposed Coding Matrix. Must have more rows than columns to solve the linear system. Must be full rank.
    x: n-array. Must be in the Code (in Ker(H)). 

    """
    n,k = tG.shape 
    
    if n < k:
        raise ValueError('Coding matrix G must have more columns than rows to solve the linear system on v\': G\'v\' = x\'')
    
                         
    rtG, rx = GaussElimination(tG,x)
    
    rank = sum([a.any() for a in rtG])
    if rank!= k:
        raise ValueError('Coding matrix G must have full rank = k to solve G\'v\' = x\'')
            
    message=np.zeros(k).astype(int)

    message[k-1]=rx[k-1]
    for i in reversed(range(k-1)):
        message[i]=abs(rx[i]-BinaryProduct(rtG[i,list(range(i+1,k))],message[list(range(i+1,k))]))

    return message



##  3 -Satistiques:  Test du décodage 
On commence par générer une matrice de parité régulière et sa matrice de codage après avoir fixé les paramètres.


In [45]:
TestDecodage = 1
if TestDecodage:   
    n= 300
    d_v = 3
    d_c  = 4
    H = RegularH(n,d_v,d_c)
    H,tG = CodingMatrix_systematic(H)
    tGs = sparse.csr_matrix(tG)
    Hs = sparse.csr_matrix(H)
k = tG.shape[1]


On teste le décodage en transmettant 10 messages aléatoires et comparant le message aléatoire à k bits envoyé v_envoye au vecteur v_recu obtenu après décodage complet. On récupère dans la liste "erreurs" le nombre de bits erronés pour chaque message. La variable "good" filtre cette liste pour n'en garder que les messages décodés sans erreurs et enregistre sa taille, c'est-à-dire le nombre de décodages réussis.

On utilise ici une matrice assez large pour montrer l'efficacité du format csr.

### 3 - 1 Test du décodage BP
Pour un canal avec un bruit modélisé par un bruit gaussien avec un SNR = 8db 

Matrices classiques:

In [46]:
if 1:    
    snr = 8
    max_iter = 1
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_BP(H,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_envoye-v_recu)))
    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de décodage classique:",t)

Le taux de décodage réussi est: 100%

 Temps de décodage classique: 1.1923930644989014


Sparse matrices:

In [47]:
if 1:    
    max_iter = 1
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_BP(Hs,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_envoye-v_recu)))
    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de décodage sparse:",t)

Le taux de décodage réussi est: 100%

 Temps de décodage sparse: 0.935791015625


Pour un canal avec un bruit modélisé par un bruit gaussien avec un SNR = 0db (Variance =1), therefore we need more iterations:

In [64]:
if 1:    
    snr = 0
    max_iter = 5
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_BP(H,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_envoye-v_recu)))
    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de décodage classique:",t)

Le taux de décodage réussi est: 50%

 Temps de décodage classique: 2.242175817489624


Sparse matrices:

In [49]:
if 1:    
    max_iter=5
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_BP(Hs,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_envoye-v_recu)))

    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de décodage sparse:",t)

Le taux de décodage réussi est: 60%

 Temps de décodage sparse: 2.653632879257202


### 3_2 Test du décodage full-log BP

Pour un bruit gaussien avec SNR = 8 db.

In [60]:
if 1:    
    snr = 8
    max_iter = 1
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_logBP(H,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_recu-v_envoye)))
    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de codage/décodage classique:",t)


Le taux de décodage réussi est: 100%

 Temps de codage/décodage classique: 0.970768928527832


Sparse matrices: 

In [61]:
if 1:    
    max_iter=1
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_logBP(Hs,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_envoye-v_recu)))
    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de codage/décodage sparse:",t)

Le taux de décodage réussi est: 100%

 Temps de codage/décodage sparse: 0.7703440189361572


Pour un canal avec un bruit modélisé par un bruit gaussien avec snr = 0, on a donc besoin de plus d'une itération:

In [62]:
if 1:    
    snr = 0
    max_iter = 6
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_logBP(H,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_recu-v_envoye)))
    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de codage/décodage classique:",t)


Le taux de décodage réussi est: 70%

 Temps de codage/décodage classique: 2.1588029861450195


Sparse matrices:

In [63]:
if 1:    
    max_iter=6
    erreurs=[]
    t = time()
    for i in range(10): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_logBP(Hs,y,snr,max_iter)
        v_recu = DecodedMessage(tG,x)
        erreurs.append(sum(abs(v_envoye-v_recu)))
    t = time() - t
    good = len([i for i in erreurs if i==0])*10 #Nombre de messages bien décodés
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de codage/décodage sparse:",t)

Le taux de décodage réussi est: 80%

 Temps de codage/décodage sparse: 2.1334900856018066


BP et Log_BP sont-ils équivalents ? En utilisant H et Hsparse ? 

In [54]:
if 1:    
    max_iter=6
    erreurs=[]
    t = time()
    for i in range(100): 
        v_envoye,y = Coding_random(tG,snr) 
        x = Decoding_logBP(H,y,snr,max_iter)
        xx = Decoding_BP(Hs,y,snr,max_iter)
        erreurs.append(sum(abs(xx-x)))
    t = time() - t
    good = len([i for i in erreurs if i==0]) #Nombre de messages égaux 
    print("Le taux de décodage réussi est: {}%".format(good))
    print("\n Temps de codage/décodage sparse:",t)

Le taux de décodage réussi est: 100%

 Temps de codage/décodage sparse: 35.69674587249756


# 4 - Applications: 